# ChatGPT and OpenAI in Python
<!-- requirement: images/llm.png -->
<!-- requirement: images/apikey_1.png -->
<!-- requirement: chatgpt_template.env -->
<!-- requirement: data/ -->

ChatGPT is an advanced AI language model created by OpenAI. With a vast range of training data, it is designed to generate human-like text responses based on prompts and inquiries. Its capabilities span numerous topics, making it a very useful tool for generating natural language content.

It can perform a wide range of tasks, including:
- **Answering Questions**: provides information and answers on a wide variety of topics such as general knowledge, science, history, technology, etc.
- **Language Translation**: translates text from one language to another.
- **Text Generation**: generates human-like text for various purposes, such as writing essays, stories, poems, or even code snippets.
- **Spell Checking and Grammar Correction**: helps identify and correct spelling mistakes or grammatical errors in text.
- **Sentiment Analysis**: analyzes the sentiment expressed in a piece of text, determining whether it is positive, negative, or neutral.
- **Summarization**: generates concise summaries of long articles or documents, providing the key points and main ideas.
- **Content Creation**: helps generate ideas and content for creative writing, marketing copy, product descriptions, and more.
- **Conversational Agent**: engages in interactive conversations, answering questions, providing suggestions, and engaging in dialogues on various topics.


The text above is a testimony to the tasks ChatGPT can perform, as it was generated by ChatGPT when asked to describe itself and what it can be used for. If you haven't yet, feel free to play around with the [web version of ChatGPT](https://chat.openai.com/) and see for yourself what it can (and can not) do. As of the time of this writing the free web version of ChatGPT runs a GPT-3.5 model, but OpenAI has developed an upgraded GPT-4 version, which is at this moment available under paid access through ChatGPT Plus.


For the purpose of this demonstration, we will use ChatGPT on yet another task - extracting information from a bunch of legal documents.

## Example: extracting information from gift agreements

We have a bunch of legal files, more precisely gift agreements that contain information about who gifted whom what. For  each agreement we would like to extract the names of the donor and recipient as well as what was gifted. If you browse through the files, you will see that all of the information is there, but acomplishing this task by hand would be tedious. We will instead use ChatGPT to help us.

We will start by using the web version of ChatGPT, so let's do that first. We can give it a prompt like "Extract who gifted whom what in the following agreement." and paste the text of one of the gift agreements.

As you can probably see it works well and gives us the information we want from all the different gift agreements. 

How does it do that?

## A machine learning model behind the magic

ChatGPT, or GPT (Generative Pre-trained Transformer), is a large language model that uses a deep learning architecture called a Transformer. Transformers are a type of deep learning architecture that have significantly advanced natural language processing (NLP) tasks. They were introduced in the groundbreaking paper "Attention Is All You Need" by Vaswani et al. in 2017 and have since become widely used in various domains, including language translation, text generation, question answering, and more. 

Going into the details of the model is not the focus of this demonstration, so let's just highlight that such models take user input together with past reponses the model generated to predict the next response/output. Transformer architecture consists of multiple layers of self-attention and feed-forward neural networks. This architecture enables the model to capture complex patterns and dependencies in the input text.

![LLM](images/llm.png)

GPT is trained on a massive amount of text data from the internet, including books, articles, websites, and more. This diverse dataset helps the model learn grammar, facts, reasoning abilities, and even some level of common sense. Before training, the text is divided into smaller units called tokens. Tokens can be as short as one character or as long as one word. This tokenization process allows the model to process and understand the text more efficiently.

During the pre-training phase, GPT learns to predict the next token in a sequence of tokens. It tries to understand the context and generate meaningful predictions based on the patterns it has learned from the training data.

After pre-training, the model undergoes a process called fine-tuning. It is further trained on more specific tasks with labeled examples. Fine-tuning allows the model to adapt to a particular use case, such as chat-based conversations.


## OpenAI API - using ChatGPT programmatically

Now that we have convinced ourselves that we can solve our task with ChatGPT and have seen a bit about how it works, we would like to use it programmatically. Using the web version is fine but has some limitations, for example if we want to integrate GPT's capabilities into other applications or we simply want to process a large amount of gift contracts, we don't want to do this by copy pasting to the web interface. We'd prefer to use the API.

OpenAI provides a [`openai`](https://github.com/openai/openai-python) Python library that will enable us to do this easily. All we need to do is make sure we have the library installed and we will need an API key. If you do no have yours yet, you can go to [this website](https://platform.openai.com/account/api-keys) and generate you API key. In case you haven't generated one yet, you will see the following:
![generateAPIkey 1](images/apikey_1.png)

After clicking on "Create new secret key", you can give it a name and a new API key will be generated. Make sure to copy it and keep it safe on your computer. We recommend putting it into a .env file. We created a `chatgpt_template.env` file for you here, so you can see what the file should look like. Fill your API key in the template and rename the file to `chatgpt.env`. Now we can load the key as an environment variable as you see below. 

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('chatgpt.env') 

openai.api_key  = os.getenv('OPENAI_API_KEY')

Now we are ready to make some requests. Note that if you do not add any billing options to your OpenAI account, you are using the free trial, which has a limited amount of free trial credits. You can check your usage [here](https://platform.openai.com/account/usage).

Let's test that this works by using a simple prompt.

In [15]:
simple_prompt = 'How much is 2 + 2?'
response = openai.Completion.create(engine='text-davinci-003',  # GPT-3
                                    temperature=0, # degree of randomness of the output           
                                    prompt=simple_prompt
                                   )

Looks like it worked. Let's investigate the response. Notice that the output we are looking for is burried in field "choices".

In [16]:
print(response.choices[0].text.strip())

2 + 2 = 4


If we want just the result (i.e. 4), we should tell it that in the prompt. Try running above with prompt "How much is 2 + 2? Give me just the result".

We are now ready to use the API on our task of extracting info from the gift contracts. Let's write two convenience functions first.

In [2]:
def generate_response(prompt, max_tokens=500):
    """
    Query OpenAI API to get response.
    """

    response = openai.Completion.create(engine='text-davinci-003',  # GPT-3
                                        temperature=0, # degree of randomness of the output
                                        prompt=prompt,
                                        max_tokens=max_tokens
                                       )
    
    return response.choices[0].text.strip()

In [3]:
def generate_prompt(gift_contract_text):
    """
    Create prompt that gets sent to OpenAI API.
    """

    prompt = f'''Extract just the donor, recipient and the exact gift from 
                 the contract. Give the result as JSON with fields Donor, 
                 Recipient and Gift. If there are several gifts break it 
                 into a simple list of strings. {gift_contract_text}'''
    
    return prompt

We decided to ask ChatGPT to give us the results in the form of JSON so we could easily process the output futher if we wanted to.

Let's try it on one contract.

In [17]:
with open('data/Gift_Agreement_2431.txt', 'r') as f:
    contract_text = f.read()

In [18]:
output = generate_response(generate_prompt(contract_text))

In [19]:
import json
json.loads(output)

{'Donor': 'Michael Thompson',
 'Recipient': 'Sarah Roberts',
 'Gift': ['$20,500 Cash', 'Vintage armchair']}

Looks good, we got what we wanted. We can now process all the gift agreements.

In [24]:
from glob import glob

files = glob('data/*.txt')

In [26]:
def process_agreement(filename):
    with open(filename, 'r') as f:
        contract_text = f.read()
    
    output = generate_response(generate_prompt(contract_text))
    
    return json.loads(output)

In [28]:
processed_agreements = [process_agreement(f) for f in files]

## ChatGPT Limitations

We've seen examples where ChatGPT can be really useful and can make various NLP tasks easy. But the tool has serious limitations and potential dangers that make it unsuitable for certain types of tasks. There are legal and ethical issues related to copyright, privacy, misuse, bias, and transparency that this new technology raises. Here are some of the known issues the tool has: 

- Propensity for generating plausible-sounding but incorrect answers.
- Limited knowledge and outdated information: ChatGPT's knowledge is based on pre-existing data up until September 2021. It might not have access to the most recent information or developments in various fields.
- Bias and inappropriate responses: The training data used to develop ChatGPT contains biases present in the source material. Consequently, the model may generate biased or inappropriate responses, especially when prompted with sensitive or controversial topics.
- Lack of contextual understanding: ChatGPT operates on a turn-by-turn basis and lacks a long-term memory of the conversation. It may sometimes provide inconsistent or incoherent responses, especially when presented with complex or multi-turn queries.
- Sensitivity to input phrasing: The model can be highly sensitive to the way a question is phrased. Slight rephrasing of the same question may yield different responses, which can be problematic when seeking consistent and accurate information.
- Security and privacy concerns: Sharing personal, sensitive, or confidential information with ChatGPT poses potential risks. As an AI model, it does not have inherent privacy or security safeguards, and data shared with it could be stored or used in unintended ways.

*Copyright &copy; 2023 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*